### IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import os
import logging
import yaml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

### IMPORT CONFIG

In [2]:
with open("../config/keras_config.yaml", 'r') as stream:
    kerasgrid_config = yaml.safe_load(stream)

In [3]:
def convert_yaml2grid(yaml):
    
    grid = [tuple([tuple(yaml[k1][k2]) for k2 in list(yaml[k1])]) for k1 in list(yaml)]
    
    return grid

In [4]:
kerasgrid_config['grid_params']['network'] = convert_yaml2grid(kerasgrid_config['grid_params']['network'])

### SETUP LOGGER

In [5]:
logging._warn_preinit_stderr = 0
logger = logging.getLogger()
# create file handler that logs debug and higher level messages
_file_path = '../logs/'
_file_name = pd.Timestamp.utcnow().strftime('%Y%m%d_%H%M_')+'notebook.log'
fh = logging.FileHandler(_file_path + _file_name)
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
fh.setFormatter(formatter)
# add the handlers to logger
logger.addHandler(ch)
logger.addHandler(fh)
logger.setLevel(logging.DEBUG)

### AUXILIARY FUNCTIONS

In [6]:
def create_model(input_layer_shape=None,
                 network=((10, 'relu', 0.1),(1,'linear',0)), 
                 optimizer='adam'):
    n = len(network)
    model = Sequential()
    model.add(Dropout(network[0][2]))
    model.add(Dense(network[0][0], input_dim=input_layer_shape, activation=network[0][1]))
    if n>1:
        for _layer in range(1,n):
            model.add(Dropout(network[_layer][2]))
            model.add(Dense(network[_layer][0], activation=network[_layer][1]))

    model.compile(loss='mae', optimizer=optimizer, metrics=['accuracy']) 
    return model


def generate_callbacks():
    
    early = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=10, 
                      verbose=0, mode='auto', baseline=None, restore_best_weights=False)
    reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, 
                             verbose=0, mode='auto', min_delta=0.001, cooldown=0, min_lr=0)
    
    return [early, reducelr]

### MAIN CLASS OBJECT

In [7]:
class gridSearchKerasNN(object):
    
    def __init__(self, config=None):
        
        if config is None:
            logger.info('No default config dictionaty was loaded')
        else:
            self.set_config_params(config)
            self._config = config
            logger.info('Default config dictionaty was initialized')
        
        return
    
    def set_grid_params(self, params):
        
        self._grid_params = params
        
        return
    
    def set_callbacks(self, callbacks_function):
        
        self._generate_callbacks = callbacks_function
        
        return
    
    def set_cv_params(self, params):
        
        self._cv_params = params
        
        return
    
    def set_config_params(self, config=None):
        
        if config is None:
            logger.info('Default config parameters will be used')
        else:
            self.set_grid_params(config['grid_params'])
            self.set_callbacks(config['keras_callbacks'])
            self.set_cv_params(config['cv_params'])
            logger.info('Passed config parameters will be used')
        
        return
    
    def initialize_network(self, network_function):
        
        self._model = KerasRegressor(build_fn=network_function)
        
        return
    
    def initialize_grid(self):
        
        _cv = self._cv_params['cv']
        _verb = self._cv_params['verb']
        _scoring = self._cv_params['scoring']
        _n_jobs = self._cv_params['n_jobs']
        
        self._grid = GridSearchCV(estimator=self._model, 
                                  param_grid=self._grid_params, 
                                  scoring=_scoring, 
                                  cv=_cv, 
                                  verbose = _verb,
                                  n_jobs = _n_jobs)
        logger.info('GridSearchCV object is instantiated')
        
        return
    
    def fit(self, X_train, y_train, X_test, y_test):
        
        self._grid_params['input_layer_shape'] = [X_train.shape[1]]
        
        self._grid_results = self._grid.fit(X_train, y_train, 
                                     validation_data=(X_test,y_test), 
                                     callbacks=self._generate_callbacks(), 
                                     verbose=0)
        
        print("Best: %f using %s" % (self._grid_results.best_score_, self._grid_results.best_params_))
        
        return
    
    def predict(self, X_test, y_test):
        
        y_pred = self._grid.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        logger.info('MAE on test: {:.2f}, RMSE on test: {:.2f}'.format(mae, rmse))
        
        return
    
    def load(self):
        
        return
    
    def dump(self):
        
        return

### EXAMPLE: BOSTON HOUSING DATASET

In [8]:
# PREPROCESSING

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [9]:
%%time
# TRAIN
gridKeras = gridSearchKerasNN(kerasgrid_config)
gridKeras.set_callbacks(generate_callbacks)
gridKeras.initialize_network(create_model)
gridKeras.initialize_grid()
gridKeras.fit(X_train, y_train, X_test, y_test)

I0102 11:29:04.896558 140657430177600 <ipython-input-7-725e9a3cac19>:40] Passed config parameters will be used
2020-01-02 11:29:04,896 - INFO - Passed config parameters will be used
I0102 11:29:04.898649 140657430177600 <ipython-input-7-725e9a3cac19>:10] Default config dictionaty was initialized
2020-01-02 11:29:04,898 - INFO - Default config dictionaty was initialized
I0102 11:29:04.900022 140657430177600 <ipython-input-7-725e9a3cac19>:63] GridSearchCV object is instantiated
2020-01-02 11:29:04,900 - INFO - GridSearchCV object is instantiated


Fitting 4 folds for each of 8 candidates, totalling 32 fits
[CV] batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=adam 
[CV]  batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=adam, total=   2.1s
[CV] batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=adam 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV]  batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=adam, total=   1.7s
[CV] batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=adam 
[CV]  batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=adam, total=   1.9s
[CV] batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=adam 
[CV]  batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=adam, total=   2.2s
[CV] batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=sgd 
[CV]  batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=sgd, total=   1.9s
[CV] batch_size=32, epochs=50, input_layer_shape=13, network=((10, 'relu', 0.1), (1, 'linear', 0)), optimizer=sgd 
[CV]  batch_siz

[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:  1.3min finished


Best: -2.830687 using {'batch_size': 32, 'epochs': 50, 'input_layer_shape': 13, 'network': ((20, 'relu', 0.1), (10, 'relu', 0.1), (1, 'linear', 0)), 'optimizer': 'sgd'}
CPU times: user 1min 31s, sys: 4.12 s, total: 1min 35s
Wall time: 1min 18s


In [10]:
# TEST
gridKeras.predict(X_test,y_test)

I0102 11:30:23.332768 140657430177600 <ipython-input-7-725e9a3cac19>:85] MAE on test: 3.53, RMSE on test: 4.78
2020-01-02 11:30:23,332 - INFO - MAE on test: 3.53, RMSE on test: 4.78
